## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [111]:
import pandas as pd
client_pay_ability = pd.read_csv('/datasets/data.csv')
print(client_pay_ability)
client_pay_ability.info()


       children  days_employed  dob_years education  education_id  \
0             1   -8437.673028         42    высшее             0   
1             1   -4024.803754         36   среднее             1   
2             0   -5623.422610         33   Среднее             1   
3             3   -4124.747207         32   среднее             1   
4             0  340266.072047         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1   -4529.316663         43   среднее             1   
21521         0  343937.404131         67   среднее             1   
21522         1   -2113.346888         38   среднее             1   
21523         3   -3112.481705         38   среднее             1   
21524         2   -1984.507589         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     0   
1       женат / замужем        

### Вывод

Всего в таблице 12 столбцов, при этом количество значений в столбцах 'days_employed', 'total_income' различное, значит, есть 'пропущенные значения'. Пять столбцов с типом данных - 'строка', еще пять - 'целые числа', оставшиеся два - 'вещественные числа'. Это отрицательные числа в днях трудового стажа (необходимо уточнить у коллег из кредитного отдела, возможно, это перерерывы в стаже или маркировка выхода на пенсию) и дроби, показывающие ежемесячный доход. Встречаются строки разного регистра и категории, дублированные по смыслу ('purpose'), это может быть неудобным для фильтрации данных.
Каждая строка таблицы позволяет узнать как о семейном, материальном и социальном положении клиента, так и о его финансовых целях.
Необходимо удалить 'пропущенные значения', 'дубликаты'. Для решения задачи будут важны столбцы 'children', 'family_status', 'debt'.

Описание данных
•	children — количество детей в семье
•	days_employed — общий трудовой стаж в днях
•	dob_years — возраст клиента в годах
•	education — уровень образования клиента
•	education_id — идентификатор уровня образования
•	family_status — семейное положение
•	family_status_id — идентификатор семейного положения
•	gender — пол клиента
•	income_type — тип занятости
•	debt — имел ли задолженность по возврату кредитов
•	total_income — ежемесячный доход
•	purpose — цель получения кредита


## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 2. Предобработка данных

### Обработка пропусков


In [112]:
client_pay_ability.isnull().sum()
client_pay_ability['days_employed'] = client_pay_ability['days_employed'].fillna('0')
client_pay_ability['total_income'] = client_pay_ability['total_income'].fillna('0')
client_pay_ability.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Проверим таблицу с помощью двух последовательных методов: поиск на наличие пропусков и их суммирование. Как мы и предполагали, пропущены значения в столбцах 'days_employed' и 'total_income', клиенты банка могут об этом не сообщать, но для кредитного отдела - это архиважная информация. Возможно, это данные по клиентам в категории 'пенсионеры'. Нужно сообщить об этом и коллегам и помочь разобраться. Пока для целей данной задачи заменим пропущенные значения в обоих столбцах на значение '0', чтобы не удалить лишнюю информацию, и проверим еще раз на наличие пропусков.

### Замена типа данных

In [113]:
client_pay_ability['days_employed'] = client_pay_ability['days_employed'].astype('int')
client_pay_ability['total_income'] = client_pay_ability['total_income'].astype('int')
client_pay_ability.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Отрицательные дробные числа в столбце 'days_employed' и положительные дробные числа в столбце 'total_income' выглядят громоздко и могут помешать корректной работе методов, поэтому изменим их тип данных с 'float' на 'integer'. Проверяем, что у нас получилось. Теперь у нас только два типа данных. Будет некорректно заменять отрицательные числа на положительные, т.к.это нарушит логику данной таблицы.

### Обработка дубликатов

In [131]:
client_pay_ability.duplicated().sum()
client_pay_ability.drop_duplicates(inplace = True)
client_pay_ability['education'] = client_pay_ability['education'].str.lower()
#print(client_pay_ability['education'].value_counts())
client_pay_ability['family_status'] = client_pay_ability['family_status'].str.lower()
#print(client_pay_ability['family_status'].value_counts())
#print(client_pay_ability['income_type'].value_counts())
#print(client_pay_ability['purpose'].value_counts())
client_pay_ability.duplicated().sum()

0

### Вывод


Мы обнаружили, что у нас 54 дубликата. Удаляем их, но,возможно, еще есть дубликаты по смыслу - выявляем вручную. Скорее всего, они встречаются в столбцах с типом данных 'строка'. Обработаем вручную каждый такой столбец на предмет уникальных значений.
1. В столбце 'education' очень много дубликатов из-за регистра. Переводим весь столбец в нижний регистр.
2. В столбце 'family_status' дубликатов нет, но одна из категорий начинается с заглавной буквы. Переводим весь столбец в нижний регистр.
3. В столбце 'income_type' все в порядке.
4. В столбце 'purpose' слишком много разных словоформ и словосочетаний с одинаковым смыслом. Для этого столбца выполним лемматизацию и категоризацию.
Проверяем еще раз - дубликатов нет.

### Лемматизация

In [123]:
from pymystem3 import Mystem
m = Mystem()
text = client_pay_ability['purpose'][447]
lemmas = m.lemmatize(text)
#print(lemmas)
text = client_pay_ability['purpose'][21524]
lemmas = m.lemmatize(text)
#print(lemmas)
from collections import Counter
print(Counter(lemmas))



Counter({' ': 2, 'на': 1, 'покупка': 1, 'автомобиль': 1, '\n': 1})


### Вывод

Проводим лемматизацию указанных целей кредита. Импортируем pymystem3, специальную библиотеку для русского языка. Выбираем несколько строк - 21524 - ['на', ' ', 'покупка', ' ', 'автомобиль', '\n'], 447 - ['получение', ' ', 'дополнительный', ' ', 'образование', '\n'].
Посчитаем, сколько уникальных слов в строке 21524. Получилось три слова, каждое используется 1 раз. Возможно, потребуется автоматизировать фильтры в данном столбце с максимальным количеством из 3-х слов - этого достаточно для описании цели кредита.



### Категоризация данных

In [115]:

def children_count(row):
    
    children = row['children']
    
    if children < 1:
        return 'нет детей'

    if children >= 1:
        return 'есть дети'

        
client_pay_ability['children_id'] = client_pay_ability.apply(children_count, axis=1)
#print(client_pay_ability.head(10))

def family_count(row):
    
    family_status_id = row['family_status_id']
    
    if family_status_id < 1:
        return 'в браке'

    if family_status_id >= 1:
        return 'не в браке'

        
client_pay_ability['family_id'] = client_pay_ability.apply(family_count, axis=1)
print(client_pay_ability.head(10))
                    
   


   children  days_employed  dob_years education  education_id  \
0         1          -8437         42    высшее             0   
1         1          -4024         36   среднее             1   
2         0          -5623         33   среднее             1   
3         3          -4124         32   среднее             1   
4         0         340266         53   среднее             1   
5         0           -926         27    высшее             0   
6         0          -2879         43    высшее             0   
7         0           -152         50   среднее             1   
8         2          -6929         35    высшее             0   
9         0          -2188         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M

### Вывод

Вернемся к данным столбцов 'children', 'family_status', 'debt'. Необходимо понять, влияет ли семейное положение и наличие детей на платежеспособность. Проведем категоризацию по типу данных:

1. Создадим функцию для разделения данных на категории 'есть дети' и 'нет детей'. Поскольку столбец 'children' содержит количество детей и затруднит подсчет данных. Добавим отдельный столбец 'children_id' только из двух значений.
2. Также поступим с семейным положением. В таблице 'семейный статус' несколько значений, но нас интересует, 'в браке' клиент или 'нет'. Добавим через функцию отдельный столбец 'family_id'.                             


### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [52]:
print(client_pay_ability.groupby('debt')['children_id'].value_counts())

debt  children_id
0     нет детей      13074
      есть дети       6639
1     нет детей       1064
      есть дети        677
Name: children_id, dtype: int64


### Вывод

Считаем количество имеющих и не имеющих долги по обеим категориям. Мы видим, что только третья часть всех клиентов имеет детей. В каждой категории рассчитываем процент тех, кто имеет задолженность по кредитам: в категории "нет детей" таких клиентов 8%, в категории "есть дети" их больше  - 10%. Из этого следует, что наличие детей незначительно влияет на возврат кредита, т.е. если у клиента есть дети, риск возникновения задолженности по оплате кредита немного возрастает.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [57]:
print(client_pay_ability.groupby('debt')['family_id'].value_counts())

debt  family_id 
0     в браке       11408
      не в браке     8305
1     в браке         931
      не в браке      810
Name: family_id, dtype: int64


### Вывод

Суммируя общее количество клиентов по статусам 'в браке' и 'не в браке', определяем, что 58% клиентов банка состоит в браке. При этом только 8,1% клиентов из категории 'в браке' не возвращает платежи по кредиту в срок. Чуть больше - 9,7%  - таких клиентов есть в категории 'не в браке'. Семейное положение влияет на возврат кредита, но не в значительной степени. Если клиент не женат/не замужем, риск несвоевременного возврата может быть выше, чем у клиента, который женат/замужем. Хотя это очевидно, в первом случае кредит может брать семья, т.е. два человека - в таком случае кредит вернуть легче, т.к. семейный доход может быть дифференцированным. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [160]:
#print(client_pay_ability['total_income'].mean())
#print(client_pay_ability['total_income'].median())


def income_count(row):
    
    total_income = row['total_income']
    
    if total_income < 135781:
        return 'ниже среднего дохода'

    if total_income > 135781 and total_income < 151011:
        return 'средний доход'
    
    if total_income > 151011:
        return 'выше среднего дохода'

    
client_pay_ability['income_average'] = client_pay_ability.apply(income_count, axis=1)
#print(client_pay_ability.head(10))
print(client_pay_ability.groupby('debt')['income_average'].value_counts())

debt  income_average      
0     ниже среднего дохода    9850
      выше среднего дохода    8385
      средний доход           1478
1     ниже среднего дохода     877
      выше среднего дохода     713
      средний доход            151
Name: income_average, dtype: int64


### Вывод

Дополнительно определяем средний доход клиента - 151 011 как среднее арифметическое и 135 781 как медианное значение, чтобы определить диапазон среднего дохода. Прописываем функцию для определения категорий среднего дохода, а также выше и ниже среднего дохода. Почти половина клиентов банка имеют низкие доходы, но данная категория более дисциплинированно платит по кредитам, чем клиенты со средним доходом. Соотвественно, доля должников 8,8% и 10,2%. Наиболее ответственены клиенты с высоким доходом и наименьшей долей должников - 8,5%. Уровень дохода влияет на возврат кредита в срок. Чем выше доход, тем меньше риска задержки оплаты.

###Как разные цели кредита влияют на его возврат в срок?

In [162]:
print(client_pay_ability.groupby('debt')['purpose'].value_counts())


debt  purpose                           
0     свадьба                               727
      сыграть свадьбу                       707
      на проведение свадьбы                 704
      операции с недвижимостью              620
      покупка коммерческой недвижимости     614
                                           ... 
1     получение образования                  37
      на покупку подержанного автомобиля     36
      ремонт жилью                           35
      покупка своего жилья                   34
      образование                            32
Name: purpose, Length: 76, dtype: int64


### Вывод

Дополнительно анализируем цели кредита. Больше всего берут кредиты на проведение свадеб и лучше всего их возвращают. Возникают проблемы чаще всего с кредитами на образование, покупку жилья и б/у автомобиля. Все логично, образование - длинная инвестиция, плюс не все успевают совмещать работу с учебой.

### Шаг 4. Общий вывод

Нашей задачей было разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Проанализировав данные мы выяснили, что клиенты, не имеющие детей, реже задерживают оплату кредита, а клиенты, состоящие в браке, более надежны в части своевременного возврата.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.